# Ejercicio 9.5

Usando Pandas, calcular la reserva siguiendo un método prospectivo y proyecciones anuales para un seguro de Decesos (Vida Entera), con los siguientes datos:
* Edad actual: 31
* Capital constante: 3.000
* Prima nivelada calculada a la edad de 25 años con las INM05 al 5\% ($\pi = C \cdot A_{25} / \ddot{a}_{25}$)
* Hipótesis mortalidad: INM05
* Tasa de descuento a utilizar: 1.39\%

In [1]:
import pandas as pd
import numpy as np
from pyliferisk import *
from pyliferisk.mortalitytables import INM05

tarifa  = Actuarial(nt=INM05, i=0.05)
mort = MortalityTable(nt=INM05)
edad = 32
C = 3000
i = 0.0139
prima = C * Ax(tarifa, 25) / annuity(tarifa, 25, 'w', 0)
w = mort.w
n = w - edad

s = pd.date_range(start='2019-12-31', periods=n, freq='Y')
df = pd.DataFrame(s, columns=['fecha'])
df['t'] = np.arange(0,n)
df['edad'] = df['t'] + edad
df['lx'] = df['edad'].apply(lambda x: mort.lx[x])
df['qx'] = df['lx'].diff(-1).fillna(0)/df['lx'][0]
df['sum_qx'] = df['qx'].cumsum()
df['px'] = 1 - df['sum_qx']
df['entradas'] = df['px'] * prima
df['salidas'] = df['qx'] * C
df['total'] = df['salidas'] - df['entradas']
df['factor_desc'] = df['t'].apply(lambda x: 1 / (1+ i) ** (x + 0.5))
df.head(10)

,fecha,t,edad,lx,qx,sum_qx,px,entradas,salidas,total,factor_desc
0,2019-12-31,0,32,98170.059786,0.000951,0.000951,0.999049,15.211353,2.853000,-12.358353,0.993122
1,2020-12-31,1,33,98076.700059,0.001028,0.001979,0.998021,15.195700,3.084064,-12.111636,0.979506
2,2021-12-31,2,34,97975.779135,0.001129,0.003108,0.996892,15.178514,3.386285,-11.792229,0.966078
3,2022-12-31,3,35,97864.968529,0.001216,0.004324,0.995676,15.159996,3.648626,-11.511370,0.952834
4,2023-12-31,4,36,97745.573267,0.001321,0.005645,0.994355,15.139879,3.963786,-11.176092,0.939771
5,2024-12-31,5,37,97615.864892,0.001469,0.007114,0.992886,15.117517,4.405986,-10.711531,0.926887
6,2025-12-31,6,38,97471.686259,0.001609,0.008723,0.991277,15.093012,4.828405,-10.264606,0.914180
7,2026-12-31,7,39,97313.684656,0.001722,0.010445,0.989555,15.066795,5.165542,-9.901253,0.901647
8,2027-12-31,8,40,97144.650785,0.001842,0.012287,0.987713,15.038756,5.524684,-9.514071,0.889286
9,2028-12-31,9,41,96963.864590,0.001947,0.014234,0.985766,15.009114,5.840348,-9.168766,0.877094


In [2]:
print(np.vdot(df.total, df.factor_desc).round(2))

1108.37
